# UAE Cancer Dataset — Derive Survival_Months & Run Pearson/Spearman and Chi-Square (with Cramér’s V)
**Course:** CDS 2413 — Programming for Data Science  
**CRN:** 14350  
**Project:** Cancer Outcome Classification (UAE Synthetic Records)  
**Group:** 3  
**Members:** Aisha Alshamsi (H00535685), Sara Almulla (H00532724) 

In [4]:
from pathlib import Path
import os, sys, re

possible_names = [
    "cancer_dataset_uae.csv",
    "UAE_Cancer_Patient_Dataset.csv",
    "uae_cancer.csv"
]
possible_dirs = [
    Path("."), Path("./data"), Path("./dataset"), Path("./datasets"),
    Path("/mnt/data"), Path("/content"), Path.home()
]

found_path = None
for d in possible_dirs:
    for name in possible_names:
        p = d / name
        if p.exists():
            found_path = p
            break
    if found_path:
        break

if not found_path:
    candidates = []
    for d in possible_dirs:
        if d.exists():
            for p in d.rglob("*.csv"):
                if re.search(r"(cancer|uae).*", p.name, re.I):
                    candidates.append(p)
    if candidates:
        found_path = candidates[0]

if not found_path:
    all_csvs = []
    for d in possible_dirs:
        if d.exists():
            all_csvs += list(d.rglob("*.csv"))
    raise FileNotFoundError(
        "Could not find the dataset. Looked in common locations.\n"
        "CSV files I can see:\n" + "\n".join(str(p) for p in all_csvs[:30])
        + ("\n... (list truncated)" if len(all_csvs) > 30 else "")
        + "\n\nFix: move/rename your file to 'cancer_dataset_uae.csv' in this notebook folder, "
          "or update `found_path` manually to the correct location."
    )

print("Using file:", found_path)

import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr, chi2_contingency

df = pd.read_csv(found_path)
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())

for col in ["Diagnosis_Date", "Death_Date"]:
    if col in df.columns and not np.issubdtype(df[col].dtype, np.datetime64):
        df[col] = pd.to_datetime(df[col], errors="coerce")

outcome_col = None
for c in ["Outcome", "outcome", "Status", "Patient_Status"]:
    if c in df.columns:
        outcome_col = c
        break
if outcome_col is None:
    raise KeyError("Could not find an outcome column. Expected one of: Outcome, Status, Patient_Status.")

def map_outcome(o):
    if pd.isna(o): return np.nan
    s = str(o).strip().lower()
    return "Deceased" if s in {"deceased","dead","passed","متوفى","death"} else "Not Deceased"

df["Deceased"] = df[outcome_col].map(map_outcome)

if {"Diagnosis_Date","Death_Date"}.issubset(df.columns):
    delta_days = (df["Death_Date"] - df["Diagnosis_Date"]).dt.days
    df["Survival_Months"] = delta_days / 30.44  # months approx
else:
    df["Survival_Months"] = np.nan

numeric_iv = None
for c in ["Age","age","Patient_Age"]:
    if c in df.columns:
        numeric_iv = c; break
if numeric_iv is None:
    raise KeyError("No Age column found. Update `numeric_iv` to your numeric predictor.")

mask = df["Deceased"].eq("Deceased") & df[numeric_iv].notna() & df["Survival_Months"].notna()
x = df.loc[mask, numeric_iv].astype(float)
y = df.loc[mask, "Survival_Months"].astype(float)

if len(x) >= 3:
    r_p, p_p = pearsonr(x, y)
    r_s, p_s = spearmanr(x, y, nan_policy="omit")
    def sig(p): return "statistically significant (p < 0.05)" if p < 0.05 else "not statistically significant (p ≥ 0.05)"
    def magnitude(v): 
        a=abs(v); return "weak" if a<0.3 else ("moderate" if a<0.5 else "strong")
    direction = "tends to increase" if r_p > 0 else ("tends to decrease" if r_p < 0 else "shows no linear change")

    print("\n=== A) Numeric association: "
          f"{numeric_iv} ↔ Survival_Months (deceased only, n={len(x)}) ===")
    print(f"Pearson r = {r_p:.3f}, p = {p_p:.4g} → {sig(p_p)}; effect size {magnitude(r_p)}.")
    print(f"Spearman ρ = {r_s:.3f}, p = {p_s:.4g} → {sig(p_s)}.")
    print(f"Interpretation: As {numeric_iv} increases, Survival_Months {direction}.\n")
else:
    print("\n=== A) Numeric association: Not enough deceased rows with both Age and Survival_Months ===")

stage_col = None
for c in ["Cancer_Stage","Stage","stage"]:
    if c in df.columns:
        stage_col = c; break
if stage_col is None:
    raise KeyError("No stage column found. Update `stage_col` to your stage variable name.")

ct = pd.crosstab(df[stage_col], df["Deceased"])
chi2, p_chi, dof, expected = chi2_contingency(ct)

n = ct.to_numpy().sum()
k = min(ct.shape)
cramers_v = np.sqrt((chi2 / n) / (k - 1)) if k > 1 else np.nan

def mag_v(v):
    if pd.isna(v): return "n/a"
    return "small" if v < 0.2 else ("moderate" if v < 0.35 else "strong")

print("=== B) Categorical association:", stage_col, "× Deceased ===")
print("Contingency table:\n", ct, "\n")
print(f"Chi-square χ² = {chi2:.3f}, df = {dof}, p = {p_chi:.4g} → "
      f"{'reject H0 (association exists)' if p_chi < 0.05 else 'fail to reject H0 (no evidence of association)'}")
print(f"Cramér’s V = {cramers_v:.3f} → {mag_v(cramers_v)} association")


Using file: cancer_dataset_uae.csv
Shape: (10000, 20)
Columns: ['Patient_ID', 'Age', 'Gender', 'Nationality', 'Emirate', 'Diagnosis_Date', 'Cancer_Type', 'Cancer_Stage', 'Treatment_Type', 'Treatment_Start_Date', 'Hospital', 'Primary_Physician', 'Outcome', 'Death_Date', 'Cause_of_Death', 'Smoking_Status', 'Comorbidities', 'Ethnicity', 'Weight', 'Height']

=== A) Numeric association: Age ↔ Survival_Months (deceased only, n=992) ===
Pearson r = 0.006, p = 0.859 → not statistically significant (p ≥ 0.05); effect size weak.
Spearman ρ = 0.005, p = 0.8776 → not statistically significant (p ≥ 0.05).
Interpretation: As Age increases, Survival_Months tends to increase.

=== B) Categorical association: Cancer_Stage × Deceased ===
Contingency table:
 Deceased      Deceased  Not Deceased
Cancer_Stage                        
I                  241          2178
II                 268          2768
III                317          2724
IV                 166          1338 

Chi-square χ² = 7.027, df 